### load ICE data: First part is most recent working code, second part is old code and utility functions, data verification and testing 

In [140]:
import pandas as pd
import os
import xmltodict
import json
import time
import gzip
import shutil
import collections

from datetime import datetime
from google.cloud import bigquery, storage
from datetime import datetime,timedelta
from google.api_core.exceptions import BadRequest
from dateutil.parser import parse

os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "/Users/Gil/git/ficc/data_loader/Cusip Global Service Importer-2fdcdfc4edba.json"

bq_schema = ['instrument_id',	'cusip',	'isin',	'security_typ',	'apex_asset_type',	'instrument_type',	'primary_name',	'delivery_date',	'issue_price',	'settlement_date',	'issue_date',	'outstanding_indicator',	'primary_currency_code',	'child_issue_ind',	'federal_tax_status',	'id',	'entity_level',	'type',	'type_id',	'text',	'incorporated_state_code',	'primary_name_abbreviated',	'organization_status',	'organization_type',	'org_country_code',	'country_code',	'maturity_amount',	'denom_increment_amount',	'min_denom_amount',	'accrual_date',	'bond_form',	'bond_insurance',	'coupon_type',	'current_coupon_rate',	'daycount_basis_type',	'debt_type',	'default_indicator',	'depository_type',	'first_coupon_date',	'interest_payment_frequency',	'issue_amount',	'last_period_accrues_from_date',	'maturity_date',	'next_coupon_payment_date',	'orig_principal_amount',	'original_yield',	'outstanding_amount',	'sale_type',	'settlement_type',	'principal_factor',	'principal_factort1',	'principal_factort2',	'principal_factort3',	'principal_factort4',	'previous_coupon_payment_date',	'bank_qualified',	'capital_type',	'dtcc_status',	'first_execution_date',	'formal_award_date',	'issue_key',	'issue_text',	'maturity_description_code',	'muni_security_type',	'other_enhancement_type',	'sale_date',	'series_name',	'state_tax_status',	'use_of_proceeds',	'asset_claim_code',	'purpose_class',	'purpose_sub_class',	'num',	'call_notice',	'call_timing',	'call_timing_in_part',	'extraordinary_make_whole_call',	'extraordinary_redemption',	'make_whole_call',	'mandatory_redemption_code',	'next_call_date',	'next_call_price',	'optional_redemption_code',	'par_call_date',	'par_call_price',	'date',	'amount_outstanding',	'amount_outstanding_decimal']
dict_data_type = {'instrument_id':'integer',	'cusip':'string',	'isin':'string',	'security_typ':'string',	'apex_asset_type':'integer',	'instrument_type':'integer',	'primary_name':'string',	'delivery_date':'date',	'issue_price':'numeric',	'settlement_date':'date',	'issue_date':'date',	'outstanding_indicator':'boolean',	'primary_currency_code':'string',	'child_issue_ind':'boolean',	'federal_tax_status':'integer',	'id':'integer',	'entity_level':'string',	'type':'string',	'type_id':'integer',	'text':'string',	'incorporated_state_code':'string',	'primary_name_abbreviated':'string',	'organization_status':'integer',	'organization_type':'integer',	'org_country_code':'string',	'country_code':'string',	'maturity_amount':'numeric',	'denom_increment_amount':'numeric',	'min_denom_amount':'numeric',	'accrual_date':'date',	'bond_form':'integer',	'bond_insurance':'string',	'coupon_type':'integer',	'current_coupon_rate':'numeric',	'daycount_basis_type':'integer',	'debt_type':'integer',	'default_indicator':'boolean',	'depository_type':'integer',	'first_coupon_date':'date',	'interest_payment_frequency':'integer',	'issue_amount':'numeric',	'last_period_accrues_from_date':'date',	'maturity_date':'date',	'next_coupon_payment_date':'date',	'orig_principal_amount':'numeric',	'original_yield':'numeric',	'outstanding_amount':'numeric',	'sale_type':'integer',	'settlement_type':'integer',	'principal_factor':'numeric',	'principal_factort1':'numeric',	'principal_factort2':'numeric',	'principal_factort3':'numeric',	'principal_factort4':'numeric',	'previous_coupon_payment_date':'date',	'bank_qualified':'boolean',	'capital_type':'integer',	'dtcc_status':'integer',	'first_execution_date':'timestamp',	'formal_award_date':'timestamp',	'issue_key':'integer',	'issue_text':'string',	'maturity_description_code':'integer',	'muni_security_type':'integer',	'other_enhancement_type':'integer',	'sale_date':'timestamp',	'series_name':'string',	'state_tax_status':'integer',	'use_of_proceeds':'integer',	'asset_claim_code':'integer',	'purpose_class':'integer',	'purpose_sub_class':'integer',	'num':'integer',	'call_notice':'integer',	'call_timing':'integer',	'call_timing_in_part':'integer',	'extraordinary_make_whole_call':'boolean',	'extraordinary_redemption':'boolean',	'make_whole_call':'boolean',	'mandatory_redemption_code':'integer',	'next_call_date':'date',	'next_call_price':'numeric',	'optional_redemption_code':'integer',	'par_call_date':'date',	'par_call_price':'numeric',	'date':'date',	'amount_outstanding':'integer',	'amount_outstanding_decimal':'numeric'}
bq_client = bigquery.Client()

### Functions:

In [133]:
#%%writefile ice_loader.py

def is_date(string, fuzzy=False):
    """
    Return whether the string can be interpreted as a date.

    :param string: str, string to check for date
    :param fuzzy: bool, ignore unknown tokens in string if True
    """
    try: 
        parse(string, fuzzy=fuzzy)
        return True

    except ValueError:
        return False
    
def is_float(string):
    try: 
        float(string)
        return True

    except ValueError:
        return False

#not in use: 
def get_next_instrument_xml():
    instrument_xml = ''
    while instrument_xml[-13:].find('</instrument>') == -1:
        instrument_xml += f.read(1)
        if instrument_xml[-10:].find('</payload>') != -1: return('EOF')
    if instrument_xml.find('<payload>') != -1: instrument_xml = instrument_xml.split('<payload>')[1]
    return(instrument_xml)

def get_instrument_xml(i):
    instrument_xml = xmls[i]
    #if i==0: instrument_xml = instrument_xml.split('<payload>')[1]
    return(instrument_xml+'</instrument>')

def flatten(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = k
        if isinstance(v, collections.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        elif type(v) == list:
            for elem in v:
                try:
                    keys = list(elem.keys())
                    if not is_date(elem[keys[0]], fuzzy=True) and not elem[keys[0]].isdecimal() and not is_float(elem[keys[0]]) and elem[keys[0]].find('/') == -1:
                        items.append((elem[keys[0]].lower(),elem[keys[len(keys)-1]]))
                    elif elem['@security_typ']:
                        items.append(('security_typ',elem['@security_typ']))    
                except Exception as e:
                    pass
        else:
            if new_key == '@id' and parent_key == 'instrument':
                items.append(('instrument_id',v))    
            elif new_key == '@id:':
                pass
            else:
                items.append((new_key.lower(), v))
    return dict(items)  

def data_type_casting(i_dict):
    dict_ice_instrument = i_dict
    for k,v in dict_ice_instrument.items():
        try:
            if(dict_data_type[k]=='date'):
                if v != None:
                    if len(v) == 10:
                        dict_ice_instrument[k] = v #datetime.strptime(v, '%Y-%m-%d').strftime('%Y-%m-%d')
                    elif len(v) == 20:
                        dict_ice_instrument[k] = v
                    else:
                        dict_ice_instrument[k] = None
            if(dict_data_type[k] == 'numeric' or dict_data_type[k] == 'float'):
                if v != '' and v != None:
                    dict_ice_instrument[k] = float(v)
                else: 
                    dict_ice_instrument[k] = None
            if(dict_data_type[k]=='string'):
                dict_ice_instrument[k] = v   
            if(dict_data_type[k]=='integer' and v != None):
                dict_ice_instrument[k] = int(v) 
            if(dict_data_type[k]=='boolean'):
                dict_ice_instrument[k] = (v == 'true')
            if(dict_data_type[k]=='timestamp'):
                dict_ice_instrument[k] = time.mktime(datetime.strptime(v, '%Y-%m-%dT%H:%M:%SZ').timetuple())
            elif(dict_data_type[k]=='time'):
                if v != None:
                    dict_ice_instrument[k] = datetime.strftime(datetime.strptime(v,'%H%M%S'),'%H:%M:%S')
        except Exception:
            dict_ice_instrument[k] = None

    return(dict_ice_instrument)

def add_to_bigquery(rows_to_insert):
    table_id = 'eng-reactor-287421.reference_data.ice_test'
    bq_client.insert_rows_json(table_id, rows_to_insert)  # Make an API request.
    #print(errors)
   
def stream_instruments(start,number_to_stream,number_of_instruments):
    rows_to_insert = []
    end = start+number_to_stream
    if end > number_of_instruments: end = number_of_instruments
    for n in range(start,end):
        xml_str = get_instrument_xml(n)
        instrument_dict = xmltodict.parse(xml_str)
        flat = flatten(instrument_dict)
        instrument = {bq_schema[i]:flat[bq_schema[i]] if bq_schema[i] in flat else None for i in range(len(bq_schema))}
        rows_to_insert.append(data_type_casting(instrument))
    #%time
    add_to_bigquery(rows_to_insert)
    
def get_xmls_list(gz_file_name):
    blob = bucket.get_blob(gz_file_name) #e.g. 'gsm_update_muni_APFICC_GSMF10I.35.1_1.20201221T0800-05.xml.gz')
    file = blob.download_to_filename('/tmp/temp_gz_file.xml.gz')

    f = gzip.open('/tmp/temp_gz_file.xml.gz', 'rt')
    file_str = f.read()
    clean_file = file_str.split('<payload>')[1]
    return (clean_file.split('</instrument>'))

### read update files daily from ref_data_1 bucket on cloud  storage: 

In [143]:
def get_cloud_storage_files():
    client = storage.Client() #storage_client
    bucket = client.get_bucket('ref_data_1')
    blobs = client.list_blobs('ref_data_1')
    file_list = []
    for blob in blobs:
        file_list.append(blob.name)
    return(file_list)

In [ ]:
def get_date_from_file_name: 
    

In [144]:
def get_last_data_entry():
    query = """
        SELECT MAX(upload_date) AS current_date
        FROM reference_data.ice_files_loading_track_log
    """
    query_job = bq_client.query(query)
    query_job.result()
    destination = query_job.destination
    table = bq_client.get_table(destination)
    # Download rows:
    rows = bq_client.list_rows(table, max_results=1)
    for row in rows:
        return(row["current_date"])
        break

In [78]:
xmls = get_xmls_list(file_name)
number_of_instruments = len(xmls)-1
#from tqdm.notebook import tqdm
#pbar = tqdm(total=number_of_instruments)
i = 0
number_to_stream = 4000
while i <= number_of_instruments:
    stream_instruments(i,number_to_stream,number_of_instruments)
    i+=number_to_stream
#    pbar.update(number_to_stream)
#pbar.close()

CPU times: user 7min 6s, sys: 1.09 s, total: 7min 7s
Wall time: 11min 7s


In [ ]:
def main():
    #get_last_data_entry:
    last = get_last_data_entry()
    #get_list_of_new_files:
    all_files = get_cloud_storage_files()
    push_files_to_bq (open_file, close)
    update_log

### Everything below is the OLD code:

### open the init file (~9gb)

In [65]:
#xmls = []
#f = open('gsm_init_muni_APFICC_GSMF10I.1.1_1.20201222T1700-05.xml','r')

In [ ]:
# Using Cloud Storage
#f = open('gsm_init_muni_APFICC_GSMF10I.1.1_1.20201222T1700-05.xml','r')
client = storage.Client()
bucket = client.get_bucket('ref_data_1')

blob = bucket.get_blob('gsm_init_muni_APFICC_GSMF10I.1.1_1.20201222T1700-05.xml')

#for reset of testing
file = blob.download_to_filename('temp_file')

f = open('temp_file','r')

In [64]:
#for reset of testing
f.close()

### if field name is date or number, drop the field (for now):

In [727]:


def is_date(string, fuzzy=False):
    """
    Return whether the string can be interpreted as a date.

    :param string: str, string to check for date
    :param fuzzy: bool, ignore unknown tokens in string if True
    """
    try: 
        parse(string, fuzzy=fuzzy)
        return True

    except ValueError:
        return False
    
def is_float(string):
    try: 
        float(string)
        return True

    except ValueError:
        return False

### read one instrument (byte by byte), return it as string:

In [8]:
def get_next_instrument_xml():
    instrument_xml = ''
    while instrument_xml[-13:].find('</instrument>') == -1:
        instrument_xml += f.read(1)
    if instrument_xml.find('<payload>') != -1: instrument_xml = instrument_xml.split('<payload>')[1]
    return(instrument_xml)

### function to flatten a deep dict object:

In [729]:
def flatten(d, parent_key='', sep='_'):
    items = []
    for k, v in d.items():
        new_key = k
        if isinstance(v, collections.MutableMapping):
            items.extend(flatten(v, new_key, sep=sep).items())
        elif type(v) == list:
            for elem in v:
                try:
                    keys = list(elem.keys())
                    if not is_date(elem[keys[0]], fuzzy=True) and not elem[keys[0]].isdecimal() and not is_float(elem[keys[0]]) and elem[keys[0]].find('/') == -1:
                        items.append((elem[keys[0]].lower(),elem[keys[len(keys)-1]]))
                    elif elem['@security_typ']:
                        items.append(('security_typ',elem['@security_typ']))    
                except Exception as e:
                    pass
        else:
            if new_key == '@id' and parent_key == 'instrument':
                items.append(('instrument_id',v))    
            elif new_key == '@id:':
                pass
            else:
                items.append((new_key.lower(), v))
    return dict(items)   

In [785]:
#DELETE: 
def vars_to_read(d, parent_key='', sep="']['"):
    items = []
    for k, v in d.items():
        new_key = parent_key + sep + k if parent_key else k
        if isinstance(v, collections.MutableMapping):
            items.extend(vars_to_read(v, new_key, sep=sep).items())
        elif type(v) == list:
            for elem in v:
                try:
                    keys = list(elem.keys())
                    if not is_date(elem[keys[0]], fuzzy=True) and not elem[keys[0]].isdecimal() and not is_float(elem[keys[0]]) and elem[keys[0]].find('/') == -1:
                        items.append((elem[keys[0]].lower(),elem[keys[len(keys)-1]]))
                    elif elem['@security_typ']:
                        items.append(('security_typ',elem['@security_typ']))    
                except Exception as e:
                    pass
        else:
            if new_key == '@id' and parent_key == 'instrument':
                items.append(('instrument_id',v))    
            elif new_key == '@id:':
                pass
            else:
                items.append((new_key.lower(), v))
    return dict(items)   

vars_to_read(instrument_dict)

{"instrument']['@id": '65477267',
 'cusip': '70917SRA3',
 'isin': 'US70917SRA32',
 'security_typ': 'FIXED',
 "instrument']['master_information']['instrument_master']['apex_asset_type": '3',
 "instrument']['master_information']['instrument_master']['instrument_type": '3',
 "instrument']['master_information']['instrument_master']['primary_name": 'REV BDS 2015',
 "instrument']['master_information']['instrument_master']['delivery_date": '2015-04-15',
 "instrument']['master_information']['instrument_master']['issue_price": '104.7080000',
 "instrument']['master_information']['instrument_master']['settlement_date": '2015-04-15',
 "instrument']['master_information']['instrument_master']['issue_date": '2015-04-15',
 "instrument']['master_information']['instrument_master']['outstanding_indicator": 'true',
 "instrument']['master_information']['instrument_master']['primary_currency_code": 'USD',
 "instrument']['master_information']['instrument_master']['primary_exchange": 'XOTC',
 "instrument']['m

### make a temp schema from the the fields in the XML, creating a union between x number of instruments:

In [624]:
schema = set()

In [625]:
for i in range(50):
    xml_str = get_next_instrument_xml()
    instrument_dict = xmltodict.parse(xml_str)
    flat = flatten(instrument_dict)
    #print(flat)
    schema = schema.union(set(flat.keys()))

#Code to convert dict to json:     
    #str_instrument_json = json.dumps(instrument_dict,indent=3)
    #instrument_json = json.loads(str_instrument_json)p
print(len(schema))
schema

135


{'#text',
 '@code',
 '@date',
 '@entity_level',
 '@id',
 '@num',
 '@organization',
 '@primary',
 '@status',
 '@type',
 '@type_id',
 'accrual_date',
 'additional_project_txt',
 'amount_outstanding',
 'amount_outstanding_decimal',
 'apex_asset_type',
 'asset_claim_code',
 'backed_underlying_security_id',
 'bank_qualified',
 'bond_form',
 'bond_insurance',
 'call_cav',
 'call_notice',
 'call_timing',
 'call_timing_in_part',
 'called_redemption_type',
 'capital_type',
 'child_issue_ind',
 'country_code',
 'country_of_quotation',
 'coupon_type',
 'currency_code',
 'current_coupon_rate',
 'cusip',
 'daycount_basis_type',
 'debt_type',
 'default_indicator',
 'delivery_date',
 'denom_increment_amount',
 'depository_type',
 'dtcc_status',
 'escrow_obligation_agent',
 'escrow_obligation_type',
 'eval_quotation_basis',
 'extraordinary_make_whole_call',
 'extraordinary_redemption',
 'federal_tax_status',
 'first_coupon_date',
 'first_execution_date',
 'formal_award_date',
 'incorporated_state_code

 ### sample code to get a single instrument xml, convert to dict, and flatten. Feel free to print out at the different steps along the way: 

In [784]:
xml_str = get_next_instrument_xml()
instrument_dict = xmltodict.parse(xml_str)
flat = flatten(instrument_dict)

In [781]:
instrument_dict['instrument']['master_information']['instrument_xref']['xref'][2]['@security_typ']
instrument_dict['instrument']['debt']['fixed_income']['current_coupon_rate']

'5.0000000'

In [635]:
single = [k for k in flat.keys()]

### Big query schema & type cast dict: 

In [730]:
bq_schema = ['instrument_id',	'cusip',	'isin',	'security_typ',	'apex_asset_type',	'instrument_type',	'primary_name',	'delivery_date',	'issue_price',	'settlement_date',	'issue_date',	'outstanding_indicator',	'primary_currency_code',	'child_issue_ind',	'federal_tax_status',	'id',	'entity_level',	'type',	'type_id',	'text',	'incorporated_state_code',	'primary_name_abbreviated',	'organization_status',	'organization_type',	'org_country_code',	'country_code',	'maturity_amount',	'denom_increment_amount',	'min_denom_amount',	'accrual_date',	'bond_form',	'bond_insurance',	'coupon_type',	'current_coupon_rate',	'daycount_basis_type',	'debt_type',	'default_indicator',	'depository_type',	'first_coupon_date',	'interest_payment_frequency',	'issue_amount',	'last_period_accrues_from_date',	'maturity_date',	'next_coupon_payment_date',	'orig_principal_amount',	'original_yield',	'outstanding_amount',	'sale_type',	'settlement_type',	'principal_factor',	'principal_factort1',	'principal_factort2',	'principal_factort3',	'principal_factort4',	'previous_coupon_payment_date',	'bank_qualified',	'capital_type',	'dtcc_status',	'first_execution_date',	'formal_award_date',	'issue_key',	'issue_text',	'maturity_description_code',	'muni_security_type',	'other_enhancement_type',	'sale_date',	'series_name',	'state_tax_status',	'use_of_proceeds',	'asset_claim_code',	'purpose_class',	'purpose_sub_class',	'num',	'call_notice',	'call_timing',	'call_timing_in_part',	'extraordinary_make_whole_call',	'extraordinary_redemption',	'make_whole_call',	'mandatory_redemption_code',	'next_call_date',	'next_call_price',	'optional_redemption_code',	'par_call_date',	'par_call_price',	'date',	'amount_outstanding',	'amount_outstanding_decimal']
dict_data_type = {'instrument_id':'integer',	'cusip':'string',	'isin':'string',	'security_typ':'string',	'apex_asset_type':'integer',	'instrument_type':'integer',	'primary_name':'string',	'delivery_date':'date',	'issue_price':'numeric',	'settlement_date':'date',	'issue_date':'date',	'outstanding_indicator':'boolean',	'primary_currency_code':'string',	'child_issue_ind':'boolean',	'federal_tax_status':'integer',	'id':'integer',	'entity_level':'string',	'type':'string',	'type_id':'integer',	'text':'string',	'incorporated_state_code':'string',	'primary_name_abbreviated':'string',	'organization_status':'integer',	'organization_type':'integer',	'org_country_code':'string',	'country_code':'string',	'maturity_amount':'numeric',	'denom_increment_amount':'numeric',	'min_denom_amount':'numeric',	'accrual_date':'date',	'bond_form':'integer',	'bond_insurance':'string',	'coupon_type':'integer',	'current_coupon_rate':'numeric',	'daycount_basis_type':'integer',	'debt_type':'integer',	'default_indicator':'boolean',	'depository_type':'integer',	'first_coupon_date':'date',	'interest_payment_frequency':'integer',	'issue_amount':'numeric',	'last_period_accrues_from_date':'date',	'maturity_date':'date',	'next_coupon_payment_date':'date',	'orig_principal_amount':'numeric',	'original_yield':'numeric',	'outstanding_amount':'numeric',	'sale_type':'integer',	'settlement_type':'integer',	'principal_factor':'numeric',	'principal_factort1':'numeric',	'principal_factort2':'numeric',	'principal_factort3':'numeric',	'principal_factort4':'numeric',	'previous_coupon_payment_date':'date',	'bank_qualified':'boolean',	'capital_type':'integer',	'dtcc_status':'integer',	'first_execution_date':'timestamp',	'formal_award_date':'timestamp',	'issue_key':'integer',	'issue_text':'string',	'maturity_description_code':'integer',	'muni_security_type':'integer',	'other_enhancement_type':'integer',	'sale_date':'timestamp',	'series_name':'string',	'state_tax_status':'integer',	'use_of_proceeds':'integer',	'asset_claim_code':'integer',	'purpose_class':'integer',	'purpose_sub_class':'integer',	'num':'integer',	'call_notice':'integer',	'call_timing':'integer',	'call_timing_in_part':'integer',	'extraordinary_make_whole_call':'boolean',	'extraordinary_redemption':'boolean',	'make_whole_call':'boolean',	'mandatory_redemption_code':'integer',	'next_call_date':'date',	'next_call_price':'numeric',	'optional_redemption_code':'integer',	'par_call_date':'date',	'par_call_price':'numeric',	'date':'date',	'amount_outstanding':'integer',	'amount_outstanding_decimal':'numeric'}

### sample instrument for debugging: 

In [709]:
#instrument = {bq_schema[i]:flat[bq_schema[i]] if bq_schema[i] in flat else None for i in range(len(bq_schema))}

### type cast function: 

In [731]:
def data_type_casting(i_dict):
    dict_ice_instrument = i_dict
    for k,v in dict_ice_instrument.items():
        try:
            if(dict_data_type[k]=='date'):
                if v != None:
                    if len(v) == 10:
                        dict_ice_instrument[k] = v #datetime.strptime(v, '%Y-%m-%d').strftime('%Y-%m-%d')
                    elif len(v) == 20:
                        dict_ice_instrument[k] = v
                    else:
                        dict_ice_instrument[k] = None
            if(dict_data_type[k] == 'numeric' or dict_data_type[k] == 'float'):
                if v != '' and v != None:
                    dict_ice_instrument[k] = float(v)
                else: 
                    dict_ice_instrument[k] = None
            if(dict_data_type[k]=='string'):
                dict_ice_instrument[k] = v   
            if(dict_data_type[k]=='integer' and v != None):
                dict_ice_instrument[k] = int(v) 
            if(dict_data_type[k]=='boolean'):
                dict_ice_instrument[k] = (v == 'true')
            if(dict_data_type[k]=='timestamp'):
                dict_ice_instrument[k] = time.mktime(datetime.strptime(v, '%Y-%m-%dT%H:%M:%SZ').timetuple())
            elif(dict_data_type[k]=='time'):
                if v != None:
                    dict_ice_instrument[k] = datetime.strftime(datetime.strptime(v,'%H%M%S'),'%H:%M:%S')
        except Exception:
            dict_ice_instrument[k] = None

    return(dict_ice_instrument)

In [652]:
#verify no data loss in type casting: 
before = {bq_schema[i]:flat[bq_schema[i]] if bq_schema[i] in flat else None for i in range(len(bq_schema))}
instrument = {bq_schema[i]:flat[bq_schema[i]] if bq_schema[i] in flat else None for i in range(len(bq_schema))}
after = data_type_casting(instrument)
[(key,before[key],after[key]) for key in instrument]

[('instrument_id', '245612919', 245612919),
 ('cusip', '942044TC2', '942044TC2'),
 ('isin', 'US942044TC22', 'US942044TC22'),
 ('security_typ', 'FIXED', 'FIXED'),
 ('apex_asset_type', '3', 3),
 ('instrument_type', '3', 3),
 ('primary_name', 'WATERTOWN MASS', 'WATERTOWN MASS'),
 ('delivery_date', '2019-06-26', '2019-06-26'),
 ('issue_price', '110.8970000', 110.897),
 ('settlement_date', '2019-06-26', '2019-06-26'),
 ('issue_date', '2019-06-26', '2019-06-26'),
 ('outstanding_indicator', 'true', True),
 ('primary_currency_code', 'USD', 'USD'),
 ('child_issue_ind', 'false', False),
 ('federal_tax_status', '2', 2),
 ('id', None, None),
 ('entity_level', None, None),
 ('type', None, None),
 ('type_id', None, None),
 ('text', None, None),
 ('incorporated_state_code', 'MA', 'MA'),
 ('primary_name_abbreviated', 'WATERTOWN MASS', 'WATERTOWN MASS'),
 ('organization_status', '6', 6),
 ('organization_type', '76', 76),
 ('org_country_code', 'US', 'US'),
 ('country_code', 'US', 'US'),
 ('maturity_amou

### output schema, if needed: 

In [636]:
import csv
with open('temp.csv', 'w', newline='') as myfile:
     wr = csv.writer(myfile, quoting=csv.QUOTE_ALL)
     wr.writerow(list(single))

### stream instrument rows to BQ: 

In [732]:
def add_to_bigquery(rows_to_insert):
    bq_client = bigquery.Client()
    table_id = 'eng-reactor-287421.reference_data.ice_test'
    bq_client.insert_rows_json(table_id, rows_to_insert)  # Make an API request.
    #print(errors)

### for n instruments: 1) read an instrument 2) make dict 3) flatten 4) make json 5) cast types correctly 6) write to BQ: 

In [733]:
def stream_instruments(number_to_stream):
    rows_to_insert = []
    n = number_to_stream

    for i in range(n):
        xml_str = get_next_instrument_xml()
        instrument_dict = xmltodict.parse(xml_str)
        flat = flatten(instrument_dict)
        instrument = {bq_schema[i]:flat[bq_schema[i]] if bq_schema[i] in flat else None for i in range(len(bq_schema))}
        rows_to_insert.append(data_type_casting(instrument))
    #%time
    add_to_bigquery(rows_to_insert)
    return(rows_to_insert)

Overwriting simulation.py


In [714]:
%%time
from tqdm.notebook import tqdm
pbar = tqdm(total=1000000)
rows_to_insert = ['n']
while rows_to_insert != []:
    rows_to_insert = stream_instruments(4000)
    pbar.update(4000)
pbar.close()
#while True:

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


KeyboardInterrupt: 

In [716]:
#!pip install line_profiler

     |████████████████████████████████| 51kB 2.9MB/s eta 0:00:011
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Created wheel for line-profiler: filename=line_profiler-3.1.0-cp37-cp37m-macosx_10_16_x86_64.whl size=51917 sha256=1acc9cf9b3abe74aef3ba0a7a9a629b2a598252d073097066ec07eb39114fcbd
  Stored in directory: /Users/Gil/Library/Caches/pip/wheels/28/ba/6e/e1686fd6ca7d0b0686e8fc4a7a1e3deff192a88d653756f9aa
Successfully built line-profiler


In [18]:
%load_ext line_profiler
#%reload_ext line_profiler
from ice_loader import stream_instruments
%lprun -T lprof0 -f stream_instruments stream_instruments(100)
#%lprun -T lprof0 -f get_next_instrument_xml get_next_instrument_xml()

The line_profiler extension is already loaded. To reload it, use:
  %reload_ext line_profiler


UnboundLocalError: local variable 'counter' referenced before assignment

In [793]:
file_str = f.read()

In [794]:
xmls = file_str.split('</instrument>')

In [795]:
len(xmls)
file_str = f.read()
xmls = file_str.split('</instrument>')

1088560

In [799]:
xmls[0]

'<?xml version="1.0" encoding="UTF-8"?><security_master v="47"><header><type>initialize</type><timestamp>2020-12-22T22:09:43Z</timestamp></header><payload><instrument id="245612919"><master_information><instrument_xref><xref type="CUSIP" type_id="1">942044TC2</xref><xref type="ISIN" type_id="2">US942044TC22</xref><xref id_bb_sec_num="MA WAT 5 06/15/2022" market_sector="Muni" security_typ="FIXED" security_typ2="Muni" ticker="WAT" type="Bloomberg Global Id" type_id="20" unique_id="FF000FC061C">BBG00PF3ZHK3</xref></instrument_xref><instrument_master><apex_asset_type>3</apex_asset_type><instrument_type>3</instrument_type><primary_name>GO MUN PURP LN BDS 2019</primary_name><delivery_date>2019-06-26</delivery_date><issue_price>110.8970000</issue_price><settlement_date>2019-06-26</settlement_date><issue_date>2019-06-26</issue_date><outstanding_indicator>true</outstanding_indicator><primary_currency_code>USD</primary_currency_code><primary_exchange>XOTC</primary_exchange><child_issue_ind>false

In [803]:
def get_instrument_xml(i):
    instrument_xml = xmls[i]
    if instrument_xml.find('<payload>') != -1: instrument_xml = instrument_xml.split('<payload>')[1]
    return(instrument_xml)

'<instrument id="236700846"><master_information><instrument_xref><xref type="CUSIP" type_id="1">768569DG7</xref><xref type="ISIN" type_id="2">US768569DG79</xref><xref id_bb_sec_num="CA RVBSCD 8 08/01/2022" market_sector="Muni" security_typ="FIXED" security_typ2="Muni" ticker="RVBSCD" type="Bloomberg Global Id" type_id="20" unique_id="FF000FB95CC">BBG00NWYQTY0</xref></instrument_xref><instrument_master><apex_asset_type>3</apex_asset_type><instrument_type>3</instrument_type><primary_name>GO BDS A</primary_name><delivery_date>2019-05-07</delivery_date><issue_price>120.2600000</issue_price><settlement_date>2019-05-07</settlement_date><issue_date>2019-05-07</issue_date><outstanding_indicator>true</outstanding_indicator><primary_currency_code>USD</primary_currency_code><primary_exchange>XOTC</primary_exchange><child_issue_ind>false</child_issue_ind><eval_quotation_basis>2</eval_quotation_basis><federal_tax_status>2</federal_tax_status></instrument_master><market_master><market id="239730769"